In [2]:
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import glob
import librosa

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# prompt: unzip file named Animals.zip

!unzip /content/drive/MyDrive/learning\ machine\ learning/Animals.zip


Archive:  /content/drive/MyDrive/learning machine learning/Animals.zip
   creating: Animals/
  inflating: __MACOSX/._Animals      
   creating: Animals/cat/
  inflating: __MACOSX/Animals/._cat  
  inflating: Animals/audio.ipynb     
  inflating: __MACOSX/Animals/._audio.ipynb  
  inflating: Animals/.DS_Store       
  inflating: __MACOSX/Animals/._.DS_Store  
   creating: Animals/dog/
  inflating: __MACOSX/Animals/._dog  
   creating: Animals/bird/
  inflating: __MACOSX/Animals/._bird  
   creating: Animals/.ipynb_checkpoints/
  inflating: __MACOSX/Animals/._.ipynb_checkpoints  
  inflating: Animals/cat/e3e49931_nohash_0.wav  
  inflating: __MACOSX/Animals/cat/._e3e49931_nohash_0.wav  
  inflating: Animals/cat/69f609ec_nohash_0.wav  
  inflating: __MACOSX/Animals/cat/._69f609ec_nohash_0.wav  
  inflating: Animals/cat/5c8af87a_nohash_0.wav  
  inflating: __MACOSX/Animals/cat/._5c8af87a_nohash_0.wav  
  inflating: Animals/cat/84bf12ff_nohash_0.wav  
  inflating: __MACOSX/Animals/cat/._84b

In [5]:
cat_audio_filenames = glob.glob('./Animals/cat/*.wav')
bird_audio_filenames = glob.glob('./Animals/bird/*.wav')
dog_audio_filenames = glob.glob('./Animals/dog/*.wav')

## Function for audio feature extraction

In [6]:
def load_audio(file_path):

    # Load audio file
    y, sr = librosa.load(file_path, sr=None)

    # mel-frequency cepstral coefficients
    # choosing 20 coefficients instead of the usual 13
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)

    # loading the first and second order derivatives of the
    # mel-frequency cepstral coefficients
    delta_mfccs = librosa.feature.delta(mfccs)
    delta2_mfccs = librosa.feature.delta(mfccs, order=2)

    # combining to form a single feature vector
    combined = np.vstack([mfccs, delta_mfccs, delta2_mfccs])
    return combined

## Preprocessing and Loading the feature vectors

In [7]:
cat_audios = []
bird_audios = []
dog_audios = []


# loading the audio files in their respective lists
for cat_audio_filename in cat_audio_filenames:
  cat_audio = load_audio(cat_audio_filename)
  cat_audios.append(cat_audio.reshape(-1, 60))

for dog_audio_filename in dog_audio_filenames:
  dog_audio = load_audio(dog_audio_filename)
  dog_audios.append(dog_audio.reshape(-1, 60))

for bird_audio_filename in bird_audio_filenames:
  bird_audio = load_audio(bird_audio_filename)
  bird_audios.append(bird_audio.reshape(-1, 60))

# getting the max sequence length for padding for each word
cat_audio_max_len = np.max([_.shape[0] for _ in cat_audios])
bird_audio_max_len = np.max([_.shape[0] for _ in bird_audios])
dog_audio_max_len = np.max([_.shape[0] for _ in dog_audios])


# padding and storing the audio feature vectors
# the feature vectors are 2D in nature

padded_cat_audios = []
padded_dog_audios = []
padded_bird_audios = []

Y = []
for cat_audio in cat_audios:
  padded_cat_audios.append(torch.concatenate((torch.from_numpy(cat_audio), torch.zeros(cat_audio_max_len - cat_audio.shape[0], 60)), dim=0))
  Y.append(0)

for dog_audio in dog_audios:
  padded_dog_audios.append(torch.concatenate((torch.from_numpy(dog_audio), torch.zeros(dog_audio_max_len - dog_audio.shape[0], 60)), dim=0))
  Y.append(1)

for bird_audio in bird_audios:
  padded_bird_audios.append(torch.concatenate((torch.from_numpy(bird_audio), torch.zeros(bird_audio_max_len - bird_audio.shape[0], 60)), dim=0))
  Y.append(2)

In [8]:
padded_cat_audios = torch.from_numpy(np.array(padded_cat_audios))
padded_dog_audios = torch.from_numpy(np.array(padded_dog_audios))
padded_bird_audios = torch.from_numpy(np.array(padded_bird_audios))

In [9]:
X = torch.cat((padded_cat_audios, padded_dog_audios, padded_bird_audios), dim=0)
X.shape

torch.Size([610, 32, 60])

In [10]:
Y = torch.from_numpy(np.array(Y))
Y.shape

torch.Size([610])

## Splitting the train and test data

In [11]:
from sklearn.model_selection import train_test_split

In [111]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, train_size=0.9)

## Network Definition

In [112]:
class AudioClassifierNetwork(nn.Module):

  def __init__(self, input_size, num_classes):
    super(AudioClassifierNetwork, self).__init__()
    self.conv_layer = nn.Sequential(
        # using 1D convolutions for 2D feature vectors
        nn.Conv1d(in_channels=32, out_channels=64, kernel_size=2, stride=1, padding=1),
        nn.MaxPool1d(kernel_size=3, stride=1, padding=1),
        nn.LeakyReLU(),
        nn.Conv1d(in_channels=64, out_channels=128, kernel_size=2, stride=1, padding=1),
        nn.MaxPool1d(kernel_size=3, stride=1, padding=1),
        nn.LeakyReLU(),
    )
    self.linear_layer = nn.Sequential(
        nn.Linear(in_features=7936, out_features=4096),
        nn.LeakyReLU(),
        nn.Linear(in_features=4096, out_features=2048),
        nn.LeakyReLU(),
        nn.Linear(in_features=2048, out_features=1024),
        nn.LeakyReLU(),
        nn.Linear(in_features=1024, out_features=256),
        nn.LeakyReLU(),
        nn.Linear(in_features=256, out_features=num_classes),
        nn.Softmax(dim=-1)
    )

  def forward(self, x):
    batch_size = x.shape[0]
    # print(x.shape)
    x = self.conv_layer(x)
    # print(x.shape)
    x = x.reshape(batch_size, -1)
    # print(x.shape)
    x = self.linear_layer(x.view(batch_size, -1))
    return x

In [113]:
acn = AudioClassifierNetwork(input_size=32, num_classes=3)

In [114]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(acn.parameters(), lr=6e-6) # lr is a little more than what karpathy would've wanted

In [115]:
import tqdm

# loss = loss_fn(acn(X_train), y_train)
# print(f'Initial loss : {loss}')

epochs = 1000
for epoch in tqdm.tqdm(range(epochs)):
  optimizer.zero_grad()
  loss = loss_fn(acn(X_train), y_train)
  print(f'Epoch {epoch + 1}, loss {loss}')
  loss.backward()
  optimizer.step()



# loss = loss_fn(acn(X_train), y_train)
# print(f'Final loss : {loss}')


  0%|          | 1/1000 [00:00<04:22,  3.81it/s]

Epoch 1, loss 1.1039432287216187


  0%|          | 2/1000 [00:00<03:50,  4.33it/s]

Epoch 2, loss 1.0958234071731567


  0%|          | 3/1000 [00:00<03:37,  4.58it/s]

...

Epoch 998, loss 0.5533729791641235


100%|█████████▉| 999/1000 [03:41<00:00,  4.37it/s]

Epoch 999, loss 0.5533726215362549


100%|██████████| 1000/1000 [03:41<00:00,  4.51it/s]

Epoch 1000, loss 0.5533723831176758


In [116]:
def calculate_accuracy(predictions, targets):
    """
    Calculate the accuracy for multi-class classification.

    Args:
    predictions (torch.Tensor): The model predictions (logits or probabilities) with shape (batch_size, num_classes).
    targets (torch.Tensor): The true labels with shape (batch_size).

    Returns:
    float: The accuracy as a percentage.
    """
    # Get the predicted class by finding the index of the maximum value in each row
    predicted_classes = torch.argmax(predictions, dim=1)

    # Calculate the number of correct predictions
    correct_predictions = (predicted_classes == targets).sum().item()

    # Calculate the accuracy
    accuracy = correct_predictions / targets.size(0) * 100.0

    return accuracy

# Example usage
# Assuming batch_size = 5, num_classes = 3


## Test accuracy

In [117]:
calculate_accuracy(acn(X_test), y_test)

85.24590163934425

## Train accuracy

In [118]:
calculate_accuracy(acn(X_train), y_train)

99.81785063752277